<div style="display: flex;">
  <img src="https://github.com/Jagroop-Dev/Xistence-Engine-WHO/blob/main/WHOl1.png?raw=true"
       width="300px"
       height="300px"
       style="margin-right: 50px;" />
  
  <img src="https://github.com/Jagroop-Dev/Xistence-Engine-WHO/blob/main/WHO%20LOGO.png?raw=true"
       width="300px"
       height="300px" />
</div>

# Team 5: Xistence Engine Exploratory Data Analysis
        
### By Jagroop Singh, Graciela Diwa and Joachim Boyden



## Feature Engineering

Preparing the data for linear regression modelling

## Imports

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, PowerTransformer

import statsmodels.api as sm
import statsmodels.tools

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Jagroop-Dev/Xistence-Engine-WHO/refs/heads/main/Life%20Expectancy%20Data.csv')

## Train Test Split the data

In [ ]:
def split_data(df :pd.DataFrame) -> tuple:
  df = df.copy()
  feature_cols = list(df.columns)
  feature_cols.remove('Life_expectancy')

  X = df[feature_cols]
  y = df['Life_expectancy']

  return train_test_split(X, y, test_size = 0.2, random_state = 1024, stratify=df['Region'])

In [ ]:
X_train, X_test, y_train, y_test = split_data(df)

#sanity check to make sure its all good !!

print(f'Train match: {X_train.shape[0] == y_train.shape[0]}') # true
print(f'Test match: {X_test.shape[0] == y_test.shape[0]}') # true

Train match: True
Test match: True


## Scaling Functions

We ended up using the rodust scaler, not only because it resulted in a better model, but because many of the features had long tails and many outliers and the fact that this scaler uses the median and IQR allows it to handle skewed data much better.

In [ ]:
def robust_scale(df :pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    rob = RobustScaler()
    rob.fit(df)
    return pd.DataFrame(rob.transform(df), columns=df.columns)

In [ ]:
def power_scale(df :pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    rob = PowerTransformer()
    rob.fit(df)
    return pd.DataFrame(rob.transform(df), columns=df.columns)

## VIF and Stepwise Functions
* to aid with feature selection and reduce multicolinearity

While we did use this VIF function in our exploration, we did not particulary use many insights gained from it in our final feature selection.

In [ ]:
## This a piece of code from stats.stackexchange.com

## It runs the model with all the variables.
## If any of them have a higher VIF than 5, it drops the max.
## Then it keeps going until none of them have a higher VIF than 5.
## This leaves us with a nice set of features with no collineraity

def calculate_vif(X, thresh = 5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        # this bit uses list comprehension to gather all the VIF values of the different variables
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif)) # getting the index of the highest VIF value
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc] # we delete the highest VIF value on condition that it's higher than the threshold
            dropped = True # if we deleted anything, we set the 'dropped' value to True to stay in the while loop

    print('Remaining variables:')
    print(X.columns[variables]) # finally, we print the variables that are still in our set
    return X.iloc[:, variables] # and return our X cut down to the remaining variables

The stepwise function on the other hand was very usefull in providing a baseline of features to use in our models. However, it did only select half of the regions. It doesn't make sense to only use half of a catergorical's catergories so, because as a whole they produced more signal than noise, we included all the region catergories.

In [ ]:
def stepwise_selection(X, y, threshold_in = 0.01, threshold_out = 0.05, verbose = True):
    # The function is checking for p-values (whether features are statistically significant) - lower is better
    included = [] # this is going to be the list of features we keep
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index = excluded, dtype = 'float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        # we add the feature with the lowest (best) p-value under the threshold to our 'included' list
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval)) # specifying the verbose text


        # backward step: removing features if new features added to the list make them statistically insignificant
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()

        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        # if the p-value exceeds the upper threshold, the feature will be dropped from the 'included' list
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

## THE Feature Engineering Function

* **OHE**: Region is not hierarchical, so label encoding would not be appropriate
* **Drop 'Country'**: Acts as an index with the year, so the model starts to memorise too much.
* **Logarithmic transformations**: GDP_per_capita, Population_mln and Incidents_HIV all had varying degrees of exponetial/logarithmic distributions, so they were transformed to be more linear.
* **Robust scaler**: Used due to the large skew and many outliers the features contain. This scaler is less effected by these things.
* **Add Constant**: so that the model can have an intercept.



In [ ]:
# mega function to do all the cleaning, build off smaller functions for modularity

def feature_eng(df):
    df = df.copy() # make copy

    # OHE - Country and Region - check please !
    #df = pd.get_dummies(df, columns = ['Country'], drop_first = True, prefix = 'Country', dtype = int)
    df = pd.get_dummies(df, columns = ['Region'], drop_first = True, prefix = 'Region', dtype = int)
    df.drop(columns = ['Country'], inplace = True)

    # Transformations: GDP_per_capita, Population_mln, Incidents_HIV
    df['GDP_per_capita'] = np.log(df['GDP_per_capita'] + 1) #log
    df['Population_mln'] = np.log(df['Population_mln'] + 1) #log
    df['Incidents_HIV'] = np.exp(df['Incidents_HIV']) #exp


    # Robust scale across DF
    df = robust_scale(df)

    df = sm.add_constant(df)


    return df

In [ ]:
X_train_fe = feature_eng(X_train)

In [ ]:
y_train = y_train.reset_index(drop=True)
X_train_fe = X_train_fe.reset_index(drop=True)

## Fit the inital model

The initial model produced the following performance metrics, which will serve as a baseline for comparing both the Ethical and Full models.

>* R-squared:	0.985
>* AIC:	7308.
>* BIC:	7457.
>* Condition Number:	1.00e+16
>* RMSE_Train: 1.1789084706465398
>* RMSE_Test: 1.5452869970162162

While the R-squared and training RMSE are very good, both the Condition number and the testing RMSE are too large. These are the metrics we will be trying to reduce in the next notebook.

In [ ]:
# using statsmodels linear regression

lin_reg = sm.OLS(y_train, X_train_fe[X_train_fe.columns])
results = lin_reg.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     5782.
Date:                Mon, 14 Jul 2025   Prob (F-statistic):               0.00
Time:                        14:41:28   Log-Likelihood:                -3627.9
No. Observations:                2291   AIC:                             7308.
Df Residuals:                    2265   BIC:                             7457.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   70.7384      0.090    786.748      0.000      70.562      70.915
Year                                     0.2796      0.046      6.014      0.000       0.188       0.371
Infant_deaths                           -1.9323      0.249     -7.750      0.000      -2.421      -1.443
Under_five_deaths                       -2.7338      0.221    -12.353      0.000      -3.168      -2.300
Adult_mortality                         -6.4100      0.063   -102.057      0.000      -6.533      -6.287
Alcohol_consumption                     -0.2112      0.077     -2.751      0.006      -0.362      -0.061
Hepatitis_B                             -0.1287      0.046     -2.771      0.006      -0.220      -0.038
Measles                                 -0.0027      0.050     -0.055      0.956      -0.101       0.095
BMI                                     -0.4141      0.073     -5.637      0.000      -0.558      -0.270
Polio                                    0.3179      0.093      3.436      0.001       0.136       0.499
Diphtheria                              -0.1841      0.093     -1.986      0.047      -0.366      -0.002
Incidents_HIV                         8.421e-10    4.2e-10      2.003      0.045    1.79e-11    1.67e-09
GDP_per_capita                           1.1123      0.090     12.354      0.000       0.936       1.289
Population_mln                           0.3617      0.043      8.389      0.000       0.277       0.446
Thinness_ten_nineteen_years             -0.1937      0.093     -2.088      0.037      -0.376      -0.012
Thinness_five_nine_years                 0.1279      0.093      1.369      0.171      -0.055       0.311
Schooling                                0.4697      0.098      4.799      0.000       0.278       0.662
Economy_status_Developed                 1.2576      0.079     15.969      0.000       1.103       1.412
Economy_status_Developing               -1.2576      0.079    -15.969      0.000      -1.412      -1.103
Region_Asia                              0.2207      0.113      1.950      0.051      -0.001       0.443
Region_Central America and Caribbean     2.0153      0.120     16.751      0.000       1.779       2.251
Region_European Union                   -0.6353      0.176     -3.606      0.000      -0.981      -0.290
Region_Middle East                      -0.0298      0.140     -0.212      0.832      -0.305       0.246
Region_North America                    -0.0209      0.247     -0.084      0.933      -0.506       0.464
Region_Oceania                          -0.5091      0.149     -3.410      0.001      -0.802      -0.216
Region_Rest of Europe                    0.5152      0.145      3.557      0.000       0.231       0.799
Region_South America                  

In [ ]:
y_pred = results.predict(X_train_fe[X_train_fe.columns])

rmse_train = statsmodels.tools.eval_measures.rmse(y_train, y_pred)

print(rmse_train)

1.1789084706465398


In [ ]:
X_test_fe = feature_eng(X_test)
#X_test_fe = X_test_fe[X_test_fe.columns]

In [ ]:


X_test_fe = X_test_fe[X_train_fe.columns]

y_test_pred = results.predict(X_test_fe)

rmse_test = statsmodels.tools.eval_measures.rmse(y_test, y_test_pred)
print(f"RMSE on Test Data: {rmse_test}")

RMSE on Test Data: 1.5231452564428307
